## Read the .csv and see data

### 00. Import libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

### 01. See the data we are working with

In [2]:
"""if you want to see the whole data you can use:"""
#pd.set_option(‘display.max_columns’, 100))

df = pd.read_csv("/kaggle/input/car-price-prediction/CarPrice_Assignment.csv")
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


### 02.Checking shape and if there are  values

In [3]:
df.shape

(205, 26)

In [4]:
df.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

### 03. Checking the datatypes of the columns

In [5]:
for i in df.columns:
    print(f"{i} - {df[i].dtype}")

car_ID - int64
symboling - int64
CarName - object
fueltype - object
aspiration - object
doornumber - object
carbody - object
drivewheel - object
enginelocation - object
wheelbase - float64
carlength - float64
carwidth - float64
carheight - float64
curbweight - int64
enginetype - object
cylindernumber - object
enginesize - int64
fuelsystem - object
boreratio - float64
stroke - float64
compressionratio - float64
horsepower - int64
peakrpm - int64
citympg - int64
highwaympg - int64
price - float64


### 04. Dividing categorical collumns from the numerical ones

In [6]:
num_columns = []
cat_columns = []

for i in df.columns:
    if df[i].dtype == "object":
        cat_columns.append(i)
    else:
        num_columns.append(i)

cat_columns
        

['CarName',
 'fueltype',
 'aspiration',
 'doornumber',
 'carbody',
 'drivewheel',
 'enginelocation',
 'enginetype',
 'cylindernumber',
 'fuelsystem']

### 05. Let's see how many distinct values there are in this categorical list

In [7]:
for i in cat_columns:
    print(f"{i}: {df[i].nunique()}")


CarName: 147
fueltype: 2
aspiration: 2
doornumber: 2
carbody: 5
drivewheel: 3
enginelocation: 2
enginetype: 7
cylindernumber: 7
fuelsystem: 8


### 06. Let’s also know the distribution of values in the categorical columns using value_counts() method.

In [8]:
print(df['fueltype'].value_counts())
print('------------------------------')
print(df['aspiration'].value_counts())
print('------------------------------')
print(df['doornumber'].value_counts())
print('------------------------------')
print(df['carbody'].value_counts())
print('------------------------------')
print(df['drivewheel'].value_counts())
print('------------------------------')
print(df['enginelocation'].value_counts())
print('------------------------------')
print(df['enginetype'].value_counts())
print('------------------------------')
print(df['cylindernumber'].value_counts())
print('------------------------------')
print(df['fuelsystem'].value_counts())

fueltype
gas       185
diesel     20
Name: count, dtype: int64
------------------------------
aspiration
std      168
turbo     37
Name: count, dtype: int64
------------------------------
doornumber
four    115
two      90
Name: count, dtype: int64
------------------------------
carbody
sedan          96
hatchback      70
wagon          25
hardtop         8
convertible     6
Name: count, dtype: int64
------------------------------
drivewheel
fwd    120
rwd     76
4wd      9
Name: count, dtype: int64
------------------------------
enginelocation
front    202
rear       3
Name: count, dtype: int64
------------------------------
enginetype
ohc      148
ohcf      15
ohcv      13
dohc      12
l         12
rotor      4
dohcv      1
Name: count, dtype: int64
------------------------------
cylindernumber
four      159
six        24
five       11
eight       5
two         4
three       1
twelve      1
Name: count, dtype: int64
------------------------------
fuelsystem
mpfi    94
2bbl    66
idi 

In [9]:
df['Car_Name'] = df['CarName'].str[:5]
df['Car_Name'].head(40)

0     alfa-
1     alfa-
2     alfa-
3     audi 
4     audi 
5     audi 
6     audi 
7     audi 
8     audi 
9     audi 
10    bmw 3
11    bmw 3
12    bmw x
13    bmw x
14    bmw z
15    bmw x
16    bmw x
17    bmw x
18    chevr
19    chevr
20    chevr
21    dodge
22    dodge
23    dodge
24    dodge
25    dodge
26    dodge
27    dodge
28    dodge
29    dodge
30    honda
31    honda
32    honda
33    honda
34    honda
35    honda
36    honda
37    honda
38    honda
39    honda
Name: Car_Name, dtype: object

### 07. Copying the original DataFrame 

In [10]:
df1 = df.copy()

In [11]:
df1.replace({'fueltype':{'gas':0, 'diesel':1}}, inplace=True)
df1.replace({'aspiration':{'std':0, 'turbo':1}}, inplace=True)
df1.replace({'doornumber':{'four':0, 'two':1}}, inplace=True)
df1.replace({'enginelocation':{'front':0, 'rear':1}}, inplace=True)
df1.replace({'drivewheel':{'fwd':0, 'rwd':1, '4wd':2}}, inplace=True)
df1.replace({'carbody':{'sedan':0, 'hatchback':1, 'wagon':2, 'hardtop':3, 'convertible':4}}, inplace=True)
df1.replace({'cylindernumber':{'two':2, 'three':3, 'four':4, 'five':5, 'six':6, 'eight':8, 'twelve':12}}, inplace=True)

/tmp/ipykernel_18/3792994071.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1.replace({'fueltype':{'gas':0, 'diesel':1}}, inplace=True)
/tmp/ipykernel_18/3792994071.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1.replace({'aspiration':{'std':0, 'turbo':1}}, inplace=True)
/tmp/ipykernel_18/3792994071.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavi

### 08. Let's see how our data look now

In [12]:
pd.set_option('display.max_columns', 100)
df1.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,Car_Name
0,1,3,alfa-romero giulia,0,0,1,4,1,0,88.6,168.8,64.1,48.8,2548,dohc,4,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-
1,2,3,alfa-romero stelvio,0,0,1,4,1,0,88.6,168.8,64.1,48.8,2548,dohc,4,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-
2,3,1,alfa-romero Quadrifoglio,0,0,1,1,1,0,94.5,171.2,65.5,52.4,2823,ohcv,6,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-
3,4,2,audi 100 ls,0,0,0,0,0,0,99.8,176.6,66.2,54.3,2337,ohc,4,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,5,2,audi 100ls,0,0,0,0,2,0,99.4,176.6,66.4,54.3,2824,ohc,5,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


### 09. We still got few categorical columns, let’s use the same formula we used earlier to store them in a list.

In [13]:
df1_cat_columns = []
df1_num_columns = []
for i in df1.columns:
  if df1[i].dtype == object:
    df1_cat_columns.append(i)
  else:
    df1_num_columns.append(i)


df1_cat_columns

['CarName', 'enginetype', 'fuelsystem', 'Car_Name']

### 09. Let's use LabelEncoder to convert the above columns into numbers.

In [14]:
le = LabelEncoder()
df1[df1_cat_columns] = le.fit_transform(df1_cat_columns)
df1[df1_cat_columns]

,CarName,enginetype,fuelsystem,Car_Name
0,0,2,3,1
1,0,2,3,1
2,0,2,3,1
3,0,2,3,1
4,0,2,3,1
...,...,...,...,...
200,0,2,3,1
201,0,2,3,1
202,0,2,3,1
203,0,2,3,1


In [15]:
df1.dtypes

car_ID                int64
symboling             int64
CarName               int64
fueltype              int64
aspiration            int64
doornumber            int64
carbody               int64
drivewheel            int64
enginelocation        int64
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype            int64
cylindernumber        int64
enginesize            int64
fuelsystem            int64
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
Car_Name              int64
dtype: object

### 10. Splitting data in features and labels

We want to predict the prices
The "car_id" is useless for our purposes so let's remove it

In [16]:
y = df1['price']
X = df1.drop(columns=['car_ID','price'])

### 11. Splitting data in train and test with sklearn
We will use 80% for training

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8,
                                                   shuffle=True, random_state=42)

# We are going to build 4 diferente models and compare the results
* Decision Tree Regressor
* Random Forest Regressor
* XGB Regressor
* Linear Regression

### 12. Let' start with Decision Tree Regressor

In [18]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
dtr_prediction = dtr.predict(X_test)

#Checking the metrics
dtr_mae = mean_absolute_error(y_test, dtr_prediction)
print(f"The Mse is: {dtr_mae}")

dtr_r2 = r2_score(y_test, dtr_prediction)
print(f"\n The r2_score is: {dtr_r2}")


The Mse is: 1853.2723658536586

 The r2_score is: 0.9029988276461218


### 13. Let's plot it
When you hover over the points in the plot you will be able to see what is the actual price and what is the predicted price. See below screen shot.

In [19]:
fig = px.scatter(x=y_test, y=dtr_prediction, labels={'x': 'Actual Price', 'y': 'Predicted Price'}, title='Decision Tree Regressor model Predictions' )
fig.show()

### 14. Random Forest Regressor

In [20]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
rfr_prediction = rfr.predict(X_test)

#Checking the metrics
rfr_mae = mean_absolute_error(y_test, rfr_prediction)
print(f"The Mse is: {rfr_mae}")

rfr_r2 = r2_score(y_test, rfr_prediction)
print(f"\n The r2_score is: {rfr_r2}")

The Mse is: 1289.269869918699

 The r2_score is: 0.9584426575910973


### 15. Let's plot and see if it is better

In [21]:
fig = px.scatter(x=y_test, y=rfr_prediction, labels={'x': 'Actual Price', 'y': 'Predicted Price'}, title='Random Forest Regressor model Predictions' )
fig.show()

### 16. Let's use XGboost and plot to see the results

In [22]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
xgb_prediction = xgb.predict(X_test)

#Checking the metrics
xgb_mae = mean_absolute_error(y_test, xgb_prediction)
print(f"The Mse is: {xgb_mae}")

xgb_r2 = r2_score(y_test, xgb_prediction)
print(f"\n The r2_score is: {xgb_r2}")

The Mse is: 1491.6630980373475

 The r2_score is: 0.9392755986153932


In [23]:
fig = px.scatter(x=y_test, y=xgb_prediction, labels={'x': 'Actual Price', 'y': 'Predicted Price'}, title='XGBRegressor model Predictions' )
fig.show()

### 17. For the last Model we will be trying Linear Regression

In [24]:
LR = LinearRegression()
LR.fit(X_train, y_train)
LR_prediction = LR.predict(X_test)

#Checking the metrics
LR_mae = mean_absolute_error(y_test, LR_prediction)
print(f"The Mse is: {LR_mae}")

LR_r2 = r2_score(y_test, LR_prediction)
print(f"\n The r2_score is: {LR_r2}")

The Mse is: 2278.8212872603995

 The r2_score is: 0.8319677810131373


In [25]:
fig = px.scatter(x=y_test, y=LR_prediction, labels={'x': 'Actual Price', 'y': 'Predicted Price'}, title='Linear Regression model Predictions' )
fig.show()

### 18. Now we plot all models together for a better visualization

In [26]:
df = pd.DataFrame({
    'Actual Price': y_test,
    'Linear Regression': LR_prediction,
    'Random Forest': rfr_prediction,   
    'XGBRegressor': xgb_prediction,
    'Decision Tree Regressor': dtr_prediction
})


# Transformando os dados para long format usando uma tupla de comprimento 1 para `id_vars`
df_long = df.melt(id_vars=('Actual Price',), var_name='Model', value_name='Predicted Price')


fig = px.scatter(df_long, x='Actual Price', y='Predicted Price', color='Model', 
                 labels={'x': 'Actual Price', 'y': 'Predicted Price'}, 
                 title='Model Predictions Comparison')
fig.show()


/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

